# Libraries

In [32]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from prettytable import PrettyTable

# Constants

In [33]:
FOLDER_PATH = './gas_consumption/'
FILE_EXSTENSION = '.json'
GAS_PRICE = 20

## Showing Results
**Important**: the cost is computed taking in account the gas price equal to 20. <br> So the formula to computed it is the following: <br> <br>ETH_PRICE = GAS_PRICE * TOTAL_GAS_USED * 1e-18 <br> <br>where 1e-18 is the conversion from Wei to Etherum,
if we use 1e-9 is the conversion from Wei to Gwei

In [44]:
def adjusting_dict(collaborator_data, device):
    len_retrieve = len(collaborator_data[device]['retrieve_fee'])
    len_send = len(collaborator_data[device]['send_fee'])

    collaborator_data[device]['retrieve_fee'] = np.sum(np.array(collaborator_data[device]['retrieve_fee'])) if collaborator_data[device]['retrieve_fee'] != [] else 0
    collaborator_data[device]['send_fee'] = np.sum(np.array(collaborator_data[device]['send_fee'])) if collaborator_data[device]['send_fee'] != [] else 0
    # computing the mean
    collaborator_data[device]['mean_retrieve_fee'] = int(collaborator_data[device]['retrieve_fee'] / len_retrieve) if len_retrieve != 0 else 0

    collaborator_data[device]['mean_send_fee'] = int(collaborator_data[device]['send_fee'] / len_send) if len_send != 0 else 0
    collaborator_data[device]['total_gas_used'] = collaborator_data[device]['retrieve_fee'] + collaborator_data[device]['send_fee'] + collaborator_data[device]['model_start_fee']

    #x = (collaborator_data[device]['total_gas_used'] * GAS_PRICE) * 1e-18
    #collaborator_data[device]['total_gas_used_ETH'] = x
def collaborator_conversion(df):
    pretty_table = PrettyTable(['Type Fee'] + df.columns.tolist())  # Set columns as headers
    for row in df.itertuples():
        pretty_table.add_row([row.Index] + list(row)[1:])
    return pretty_table

def manager_conversion(df):
    pretty_table = PrettyTable(df.columns.tolist())  # Set column names as headers
    for row in df.itertuples(index=False):
        pretty_table.add_row(row)
    return pretty_table

all_files = os.listdir(FOLDER_PATH)

# Filter files based on the pattern Name*_collaborator.json and Name*_manager.json
collaborator_files = [file for file in all_files if '_collaborator' in file]
manager_files = [file for file in all_files if '_manager' in file]

# Process files in pairs (collaborator and manager)
for collaborator_file, manager_file in zip(collaborator_files, manager_files):
    collaborator_path = os.path.join(FOLDER_PATH, collaborator_file)
    manager_path = os.path.join(FOLDER_PATH, manager_file)

    # Load JSON data from files
    with open(collaborator_path, 'r') as f_collaborator, open(manager_path, 'r') as f_manager:
        collaborator_data = json.load(f_collaborator)
        manager_data = json.load(f_manager)

    for device in collaborator_data.keys():
        adjusting_dict(collaborator_data, device)

    sum_send_aggr_weights = np.sum(np.array(manager_data['send_aggregated_weights_fee']))
    total_cost = sum_send_aggr_weights + np.sum(np.array([value for key,value in manager_data.items() if key !=  'send_aggregated_weights_fee' ]))

    manager_data['total_gas_used'] =total_cost


    #manager_data['send_aggregated_weights_fee'] = [manager_data['send_aggregated_weights_fee'][0],manager_data['send_aggregated_weights_fee'][1]]

    collaborator_df = pd.DataFrame.from_dict(collaborator_data, orient='index').T
    manager_df = pd.DataFrame.from_dict(manager_data, orient='index').T
    name_file = collaborator_file.split('_')
    # Print DataFrames
    print(f"\t\tFees for {name_file[-3]} devices and {name_file[-2]} out of battery:\n")
    collaborator_table = collaborator_conversion(collaborator_df)
    collaborator_table.title = "Collaborator"
    print(collaborator_table)
    print()
    manager_table = manager_conversion(manager_df)
    manager_table.title = "Manager"
    print("Manager:\n",manager_table)
    #manager_df.head()
    print("\n" + "=" * 40 + "\n")


		Fees for 3 devices and ['A'] out of battery:

+------------------------------------------------+
|                  Collaborator                  |
+-------------------+--------+---------+---------+
|      Type Fee     |   A    |    B    |    C    |
+-------------------+--------+---------+---------+
|    retrieve_fee   |   0    |  283788 |  307107 |
|      send_fee     | 155983 | 1555740 | 1313546 |
|  model_start_fee  | 150325 |  125507 |  134469 |
| mean_retrieve_fee |   0    |  31532  |  34123  |
|   mean_send_fee   | 155983 |  155574 |  131354 |
|   total_gas_used  | 306308 | 1965035 | 1755122 |
+-------------------+--------+---------+---------+

Manager:
 +---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                           Manager                                      